# 5G NR PUSCH Tutorial

This notebook provides an introduction to Sionna's [5G New Radio (NR) module](https://nvlabs.github.io/sionna/api/nr.html) and, in particular, the [physical uplink shared channel (PUSCH)](https://nvlabs.github.io/sionna/api/nr.html#pusch). This module provides implementations of a small subset of the physical layer functionalities as described in the 3GPP specifications [38.211](https://portal.3gpp.org/desktopmodules/Specifications/SpecificationDetails.aspx?specificationId=3213), [38.212](https://portal.3gpp.org/desktopmodules/Specifications/SpecificationDetails.aspx?specificationId=3214) and [38.214](https://portal.3gpp.org/desktopmodules/Specifications/SpecificationDetails.aspx?specificationId=3216). 


You will

- Get an understanding of the different components of a PUSCH configuration, such as the carrier, DMRS, and transport block,
- Learn how to rapidly simulate PUSCH transmissions for multiple transmitters,
- Modify the PUSCHReceiver to use a custom MIMO Detector.

## Table of Contents
* [GPU Configuration and Imports](#GPU-Configuration-and-Imports)
* [A "Hello World!" Example](#A-Hello-World-Example)
* [Carrier Configuration](#Carrier-Configuration)
* [Understanding the DMRS Configuration](#Understanding-the-DMRS-Configuration)
    * [Configuring Multiple Layers](#Configuring-Multiple-Layers)
    * [Controlling the Number of DMRS Symbols in a Slot](#Controlling-the-Number-of-DMRS-Symbols-in-a-Slot)
    * [How to control the number of available DMRS ports?](#How-to-control-the-number-of-available-DMRS-ports?)
* [Transport Blocks and MCS](#Transport-Blocks-and-MCS)
* [Looking into the PUSCHTransmitter](#Looking-into-the-PUSCHTransmitter)
* [Components of the PUSCHReceiver](#Components-of-the-PUSCHReceiver)
* [End-to-end PUSCH Simulations](#End-to-end-PUSCH-Simulations)


## GPU Configuration and Imports

In [1]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# tahtan
# Import Sionna
import sys
sys.path.append('../')
import sionna

# try:
#     import sionna
# except ImportError as e:
#     # Install Sionna if package is not already installed
#     import os
#     os.system("pip install sionna")
#     import sionna

import tensorflow as tf
# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.config.seed = 42 # Set seed for reproducible results

# Load the required Sionna components
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver, CarrierConfig, PUSCHDMRSConfig, TBConfig
from sionna.nr.utils import generate_prng_seq
from sionna.channel import AWGN, RayleighBlockFading, OFDMChannel, TimeChannel, time_lag_discrete_time_channel
from sionna.channel.utils import * 
from sionna.channel.tr38901 import Antenna, AntennaArray, UMi, UMa, RMa, TDL, CDL
from sionna.channel import gen_single_sector_topology as gen_topology
from sionna.utils import compute_ber, ebnodb2no, sim_ber, array_to_hash, create_timestamped_folders, b2b, f2f
from sionna.ofdm import KBestDetector, LinearDetector, ResourceGrid, MaximumLikelihoodDetector
from sionna.mimo import StreamManagement


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import json
from tqdm.notebook import tqdm
import itertools

## A Hello World Example

Let us start with a simple "Hello, World!" example in which we will simulate PUSCH transmissions from a single transmitter to a single receiver over an AWGN channel.

In [3]:
num_rx_ant = 8
num_tx_ant = 1
carrier_frequency = 2.55e-9

# Configure antenna arrays
ue_antenna = Antenna(polarization="single",
                polarization_type="V",
                antenna_pattern="38.901",
                carrier_frequency=carrier_frequency)

gnb_array = AntennaArray(num_rows=1,
                        num_cols=num_rx_ant//2,
                        polarization="dual",
                        polarization_type="cross",
                        antenna_pattern="38.901",
                        carrier_frequency=carrier_frequency)

channel_model_0 = CDL("A", 150e-9, carrier_frequency, ue_antenna, gnb_array, "uplink", min_speed=0.8333)

In [4]:
class MyPUSCHConfig(PUSCHConfig):
    def __init__(self):
        super().__init__(
            carrier_config=CarrierConfig(
                n_cell_id=500,
                cyclic_prefix="normal",
                subcarrier_spacing=30,
                n_size_grid=273,
                n_start_grid=0,
                slot_number=4,
                frame_number=0
            ),
            pusch_dmrs_config=PUSCHDMRSConfig(
                config_type=1,
                length=1,
                additional_position=1,
                dmrs_port_set=[0],
                n_id=0,
                n_scid=0,
                num_cdm_groups_without_data=2,
                type_a_position=2
            ),
            tb_config=TBConfig(
                channel_type='PUSCH',
                n_id=20,
                mcs_table=1,
                mcs_index=9
            ),
            mapping_type='A',
            n_size_bwp=273,
            n_start_bwp=0,
            num_layers=1,
            num_antenna_ports=1,
            precoding='non-codebook',
            tpmi=0,
            transform_precoding=False,
            n_rnti=2001,
            symbol_allocation=[0,14]
        )

pusch_config_0 = MyPUSCHConfig()
pusch_config_0.show()

Carrier Configuration
cyclic_prefix : normal
cyclic_prefix_length : 2.3437500000000002e-06
frame_duration : 0.01
frame_number : 0
kappa : 64.0
mu : 1
n_cell_id : 500
n_size_grid : 273
n_start_grid : 0
num_slots_per_frame : 20
num_slots_per_subframe : 2
num_symbols_per_slot : 14
slot_number : 4
sub_frame_duration : 0.001
subcarrier_spacing : 30
t_c : 5.086263020833334e-10
t_s : 3.2552083333333335e-08

PUSCH Configuration
dmrs_grid : shape (1, 3276, 14)
dmrs_grid_precoded : shape ()
dmrs_mask : shape (3276, 14)
dmrs_symbol_indices : [2, 11]
frequency_hopping : neither
l : [2, 11]
l_0 : 2
l_bar : [2, 11]
l_d : 14
l_prime : [0]
l_ref : 0
mapping_type : A
n : shape (819,)
n_rnti : 2001
n_size_bwp : 273
n_start_bwp : 0
num_antenna_ports : 1
num_coded_bits : 78624
num_layers : 1
num_ov : 0
num_res_per_prb : 144
num_resource_blocks : 273
num_subcarriers : 3276
precoding : non-codebook
precoding_matrix : None
symbol_allocation : [0, 14]
tb_size : 52224
tpmi : 0
transform_precoding : False

PUSC

In [5]:
# ABCD = dict()
# for Mcs in range(0,10):
#     print(Mcs)
#     ABCD[Mcs] = []
#     for Nrb in range(3,44):
       
#         pusch_config_i = MyPUSCHConfig()
#         pusch_config_i.n_size_bwp = Nrb
#         pusch_config_i.tb.mcs_index = Mcs

#         # try: 
#         PUSCHTransmitter(pusch_config_i)(1)
#         # except ValueError as e: 
#         #     print(e, 'NRB:', Nrb, 'MCS:', Mcs, 'Code rate: ', pusch_config_i.tb.target_coderate)
#         #     ABCD[Mcs].append(Nrb)

# print(ABCD)
      



Next, we test if this receiver works over a simple Rayleigh block fading channel:

In [18]:
# We need to enable sionna.config.xla_compat before we can use
# tf.function with jit_compile=True.
# See https://nvlabs.github.io/sionna/api/config.html#sionna.Config.xla_compat
sionna.config.xla_compat=True

class MyGenerator(tf.keras.layers.Layer):
    def __init__(self, pusch_config, channel_model, mode=1, dir='../Dataset'):
        super().__init__()
        # Create a PUSCH configuration with default settings
        self.dir = dir
        os.makedirs(self.dir, exist_ok=True)
        self._pusch_config = pusch_config

        self.tb_size = pusch_config.tb_size
        self.num_coded_bits = pusch_config.num_coded_bits
        self.num_subcarriers = 12*pusch_config.n_size_bwp
        self.num_symbols = pusch_config.symbol_allocation[1]
        self.dmrs_symbol_indices = pusch_config.dmrs_symbol_indices

        self.mcs = pusch_config.tb.mcs_index
        # Instantiate a PUSCHTransmitter from the PUSCHConfig
        self._pusch_transmitter = PUSCHTransmitter(pusch_config)
                
        
        rx_tx_association = np.ones([1, 1], bool)
        stream_management = StreamManagement(rx_tx_association,1)   

        detector = MaximumLikelihoodDetector(output="bit",
                                             demapping_method="maxlog",
                                             resource_grid=self._pusch_transmitter.resource_grid,
                                             stream_management=stream_management,
                                             constellation_type="qam",
                                             num_bits_per_symbol=pusch_config.tb.num_bits_per_symbol)
            
        if mode==0:
            self._pusch_receiver = PUSCHReceiver(self._pusch_transmitter,
                                                 mimo_detector=detector,
                                                 channel_estimator="perfect")
        else:
            self._pusch_receiver = PUSCHReceiver(self._pusch_transmitter)
        

        file_path = os.path.join(f'{self.dir}', f'pusch_transmiter.txt')
        with open(file_path, 'w') as f:
            self._pusch_transmitter.show(f)
    
        self._channel = OFDMChannel(channel_model,
                            self._pusch_transmitter.resource_grid,
                            return_channel=True)

        self.save_to_dataset = [*list('bcxhny'),'b_hat', 'mcs']


    def poll(self):
        existing_samples = 0
        if os.path.isfile(f'{self.dir}/b.bin'):
            existing_samples = os.path.getsize(f'{self.dir}/b.bin') * 8 // self.num_coded_bits
        return existing_samples
        
    def call(self, batch_size, ebno_db):
        if self.save_to_dataset:
            no_dir = f'{self.dir}'
            os.makedirs(no_dir, exist_ok=True)

        x, b, c = self._pusch_transmitter(batch_size, ret=['c'])
       
        n = ebnodb2no(ebno_db,
                    self._pusch_transmitter._num_bits_per_symbol,
                    self._pusch_transmitter._target_coderate,
                    self._pusch_transmitter.resource_grid)
        
        print(n)

        y, h = self._channel([x, n])

        if 'mcs' in self.save_to_dataset:
            batch_mcs = tf.repeat(self.mcs, repeats=batch_size)
            mcsbin = os.path.join(f'{no_dir}', f'mcs.bin')
            with open(mcsbin, 'ab') as file:
                file.write(tf.cast(batch_mcs, tf.uint8))

        if 'b' in self.save_to_dataset:
            bbin = os.path.join(f'{no_dir}', f'b.bin')
            with open(bbin, 'ab') as file:
                file.write(b2b(tf.pad(b, tf.constant([[0,0],[0,0],[0, self.num_coded_bits - self.tb_size]]))))
                
        if 'c' in self.save_to_dataset:
            cbin = os.path.join(f'{no_dir}', f'c.bin')
            with open(cbin, 'ab') as file:
                file.write(b2b(c))

        if 'x' in self.save_to_dataset:
            xbin = os.path.join(f'{no_dir}', f'x.bin')
            with open(xbin, 'ab') as file:
                file.write(f2f(tf.stack((tf.math.real(x), tf.math.imag(x)), axis=-1)))

        if 'h' in self.save_to_dataset:
            hbin = os.path.join(f'{no_dir}', f'h.bin')
            with open(hbin, 'ab') as file:
                file.write(f2f(tf.stack((tf.math.real(h), tf.math.imag(h)), axis=-1)))

        if 'y' in self.save_to_dataset:        
            ybin = os.path.join(f'{no_dir}', f'y.bin')
            with open(ybin, 'ab') as file:
                file.write(f2f(tf.stack((tf.math.real(y), tf.math.imag(y)), axis=-1)))

        if 'n' in self.save_to_dataset:    
            batch_n = tf.repeat(n, repeats=batch_size)
            nbin = os.path.join(f'{no_dir}', f'n.bin')
            with open(nbin, 'ab') as file:
                file.write(tf.cast((batch_n * (2 ** 6)), tf.int8))
            
        if self._pusch_receiver._perfect_csi:
            b_hat = self._pusch_receiver([y, h, n]) 
        else:
            b_hat = self._pusch_receiver([y, n])

        if 'b_hat' in self.save_to_dataset:
            b_hatbin = os.path.join(f'{no_dir}', f'b_hat.bin')
            with open(b_hatbin, 'ab') as file:
                file.write(b2b(tf.pad(b_hat, tf.constant([[0,0],[0,0],[0, self.num_coded_bits - self.tb_size]]))))

        return b, b_hat

class MyLoader(tf.keras.layers.Layer):
    def __init__(self, generator):
        super().__init__()
        # Create a PUSCH configuration with default settings
        self.dir = generator.dir

        self.tb_size = generator.tb_size
        self.num_coded_bits = generator.num_coded_bits
        self.num_subcarriers = generator.num_subcarriers
        self.num_symbols = generator.num_symbols
        self.dmrs_symbol_indices = generator.dmrs_symbol_indices
        # Instantiate a PUSCHTransmitter from the PUSCHConfig                  

        self.dataset = tf.data.Dataset.from_generator(self.load, 
                        output_signature=(
                        tf.TensorSpec(shape=(1, self.tb_size), dtype=tf.float32),
                        tf.TensorSpec(shape=(1, self.tb_size), dtype=tf.float32),
                        tf.TensorSpec(shape=(1, self.num_coded_bits), dtype=tf.float32),
                        tf.TensorSpec(shape=(1, 1, self.num_symbols, self.num_subcarriers), dtype=tf.complex64),
                        tf.TensorSpec(shape=(1, 8, 1, 1, self.num_symbols, self.num_subcarriers), dtype=tf.complex64),
                        tf.TensorSpec(shape=(1, 8, self.num_symbols, self.num_subcarriers), dtype=tf.complex64),
                        tf.TensorSpec(shape=(), dtype=tf.float32),
                        tf.TensorSpec(shape=(), dtype=tf.uint8)
                        ))       
        
        self.model_dataset = tf.data.Dataset.from_generator(self.load2trainer, 
                        output_signature=(
                        tf.TensorSpec(shape=(1, 1, self.num_symbols-2, self.num_subcarriers, 2), dtype=tf.uint8),
                        tf.TensorSpec(shape=(1, 8, self.num_symbols-2, self.num_subcarriers, 2), dtype=tf.int16),
                        tf.TensorSpec(shape=(), dtype=tf.float32),
                        tf.TensorSpec(shape=(), dtype=tf.uint8)
                        )) 

    def load(self):
        no_dir = f'{self.dir}'

        with open(os.path.join(no_dir, 'b.bin'), "rb") as bfile, \
                open(os.path.join(no_dir, 'b_hat.bin'), "rb") as b_hatfile, \
                open(os.path.join(no_dir, 'c.bin'), "rb") as cfile, \
                open(os.path.join(no_dir, 'x.bin'), "rb") as xfile, \
                open(os.path.join(no_dir, 'h.bin'), "rb") as hfile, \
                open(os.path.join(no_dir, 'y.bin'), "rb") as yfile, \
                open(os.path.join(no_dir, 'mcs.bin'), "rb") as mcsfile, \
                open(os.path.join(no_dir, 'n.bin'), "rb") as nfile:
        
            while True:
                b_batch = np.frombuffer(bfile.read(1 * self.num_coded_bits // 8), dtype=np.uint8)
                b_hat_batch = np.frombuffer(b_hatfile.read(1 * self.num_coded_bits // 8), dtype=np.uint8)
                c_batch = np.frombuffer(cfile.read(1 * self.num_coded_bits // 8), dtype=np.uint8)
                x_batch = np.frombuffer(xfile.read(1 * self.num_symbols * self.num_subcarriers * 2 * 2), dtype=np.int16)
                h_batch = np.frombuffer(hfile.read(1 * 8 * self.num_symbols * self.num_subcarriers * 2 * 2), dtype=np.int16)
                y_batch = np.frombuffer(yfile.read(1 * 8 * self.num_symbols * self.num_subcarriers * 2 * 2), dtype=np.int16)
                n_batch = np.frombuffer(nfile.read(1), dtype=np.int8)
                mcs_batch = np.frombuffer(mcsfile.read(1), dtype=np.uint8)

                if not b_batch.size: break  # Stop when reaching the end

                b = tf.cast(np.unpackbits(b_batch).reshape(1, self.num_coded_bits),tf.float32)[:,:self.tb_size]
                b_hat = tf.cast(np.unpackbits(b_hat_batch).reshape(1, self.num_coded_bits),tf.float32)[:,:self.tb_size]
                c = tf.cast(np.unpackbits(c_batch).reshape(1, self.num_coded_bits),tf.float32)
                x = (x_batch.reshape(1, 1, self.num_symbols, self.num_subcarriers, 2))
                x = tf.cast((x[..., 0] + 1j * x[..., 1]) / (2**13), tf.complex64)
                h = (h_batch.reshape(1, 8, 1, 1, self.num_symbols, self.num_subcarriers, 2))
                h = tf.cast((h[..., 0] + 1j * h[..., 1]) / (2**13), tf.complex64)
                y = (y_batch.reshape(1, 8, self.num_symbols, self.num_subcarriers, 2))
                y = tf.cast((y[..., 0] + 1j * y[..., 1]) / (2**13), tf.complex64)
                n = tf.cast(n_batch / (2**6), tf.float32)[0]
                mcs = mcs_batch[0]
                yield b, b_hat, c, x, h, y, n, mcs

    def load2trainer(self):
        no_dir = f'{self.dir}'

        with open(os.path.join(no_dir, 'c.bin'), "rb") as cfile, \
                open(os.path.join(no_dir, 'y.bin'), "rb") as yfile, \
                open(os.path.join(no_dir, 'mcs.bin'), "rb") as mcsfile, \
                open(os.path.join(no_dir, 'n.bin'), "rb") as nfile:
        
            while True:
                c_batch = np.frombuffer(cfile.read(1 * self.num_coded_bits // 8), dtype=np.uint8)
                y_batch = np.frombuffer(yfile.read(1 * 8 * self.num_symbols * self.num_subcarriers * 2 * 2), dtype=np.int16)
                n_batch = np.frombuffer(nfile.read(1), dtype=np.int8)
                mcs_batch = np.frombuffer(mcsfile.read(1), dtype=np.uint8)

                if not c_batch.size: break  # Stop when reaching the end
                c = np.unpackbits(c_batch).reshape(1, 1, self.num_symbols - 2, self.num_subcarriers, 2)
                y = (y_batch.reshape(1, 8, self.num_symbols, self.num_subcarriers, 2))
                y = tf.concat([y[:,:,0:2],y[:,:,4:12],y[:,:,12:14]],axis=2)

                n = tf.cast(n_batch[0]/(2**6),tf.float32)
                mcs = mcs_batch[0]

                yield c, y, n, mcs
generator = MyGenerator(pusch_config_0, channel_model_0, mode=0, dir='/workspaces/sionna/Dataset/CaseNo/154432424')
loader = MyLoader(generator)

XLA can lead to reduced numerical precision. Use with care.


In [ ]:
for a in range(-17,-13):
    print(generator.poll())
    print(a)
    generator(2,a)
print(generator.poll())

0
-17
tf.Tensor(47.19966, shape=(), dtype=float32)
2
-16
tf.Tensor(37.492023, shape=(), dtype=float32)
4
-15
tf.Tensor(29.780972, shape=(), dtype=float32)
6
-14
tf.Tensor(23.655863, shape=(), dtype=float32)


In [ ]:
for batch_num, (b, b_hat, c, x, h, y, n, mcs) in enumerate(loader.dataset.batch(1)):
    print(batch_num, b.shape, b_hat.shape, c.shape, x.shape, h.shape, y.shape, n.shape, mcs)

0 (1, 1, 52224) (1, 1, 52224) (1, 1, 78624) (1, 1, 1, 14, 3276) (1, 1, 8, 1, 1, 14, 3276) (1, 1, 8, 14, 3276) (1,) tf.Tensor([9], shape=(1,), dtype=uint8)
1 (1, 1, 52224) (1, 1, 52224) (1, 1, 78624) (1, 1, 1, 14, 3276) (1, 1, 8, 1, 1, 14, 3276) (1, 1, 8, 14, 3276) (1,) tf.Tensor([9], shape=(1,), dtype=uint8)
2 (1, 1, 52224) (1, 1, 52224) (1, 1, 78624) (1, 1, 1, 14, 3276) (1, 1, 8, 1, 1, 14, 3276) (1, 1, 8, 14, 3276) (1,) tf.Tensor([9], shape=(1,), dtype=uint8)
3 (1, 1, 52224) (1, 1, 52224) (1, 1, 78624) (1, 1, 1, 14, 3276) (1, 1, 8, 1, 1, 14, 3276) (1, 1, 8, 14, 3276) (1,) tf.Tensor([9], shape=(1,), dtype=uint8)


In [ ]:
for batch_num, (c, y, n, mcs) in enumerate(loader.model_dataset.batch(2)):
    print(batch_num, c.shape, y.shape, n.shape, mcs.shape)

0 (2, 1, 1, 12, 3276, 2) (2, 1, 8, 12, 3276, 2) (2,) (2,)
1 (2, 1, 1, 12, 3276, 2) (2, 1, 8, 12, 3276, 2) (2,) (2,)


In [ ]:
c, y, n, mcs

(<tf.Tensor: shape=(2, 1, 1, 12, 3276, 2), dtype=uint8, numpy=
 array([[[[[[1, 0],
            [0, 1],
            [1, 0],
            ...,
            [1, 0],
            [0, 0],
            [1, 1]],
 
           [[1, 0],
            [0, 0],
            [1, 0],
            ...,
            [0, 0],
            [0, 1],
            [0, 1]],
 
           [[0, 0],
            [0, 1],
            [1, 1],
            ...,
            [0, 1],
            [1, 0],
            [1, 1]],
 
           ...,
 
           [[1, 1],
            [0, 1],
            [1, 0],
            ...,
            [0, 1],
            [1, 0],
            [0, 0]],
 
           [[1, 1],
            [1, 1],
            [0, 1],
            ...,
            [0, 1],
            [0, 1],
            [1, 1]],
 
           [[0, 1],
            [0, 1],
            [0, 0],
            ...,
            [0, 1],
            [1, 1],
            [1, 1]]]]],
 
 
 
 
        [[[[[1, 1],
            [1, 1],
            [0, 1],
          

In [ ]:
aafas = PUSCHTransmitter(pusch_config_0)


In [ ]:
aafas._pusch_configs[0].tb.show()

Transport Block Configuration
channel_type : PUSCH
mcs_index : 9
mcs_table : 1
n_id : 20
num_bits_per_symbol : 2
target_coderate : 0.6630859375
tb_scaling : 1.0



In [ ]:
compute_ber(*model(2,-17.))

<tf.Tensor: shape=(), dtype=float64, numpy=0.07140174278846154>

In [ ]:
compute_ber(*generator(2,-9.5))

<tf.Tensor: shape=(), dtype=float64, numpy=0.1247702205882353>

In [12]:
generator = MyGenerator(pusch_config_0, channel_model_0, dir = '/workspaces/sionna/Dataset/2025-02-13/10-13-52/CDLA_speed10_ds150.0')
loader = MyLoader(generator)

In [13]:
generator.poll()

28

In [14]:
for batch_num, (c, y, n, mcs) in enumerate(loader.model_dataset.batch(2)):
    print(batch_num, c.shape, y.shape, n.shape, mcs.shape, n, mcs)

0 (2, 1, 1, 12, 3276, 2) (2, 1, 8, 12, 3276, 2) (2,) (2,) tf.Tensor([0.859375 0.859375], shape=(2,), dtype=float32) tf.Tensor([5 5], shape=(2,), dtype=uint8)
1 (2, 1, 1, 12, 3276, 2) (2, 1, 8, 12, 3276, 2) (2,) (2,) tf.Tensor([0.859375 0.859375], shape=(2,), dtype=float32) tf.Tensor([5 5], shape=(2,), dtype=uint8)
2 (2, 1, 1, 12, 3276, 2) (2, 1, 8, 12, 3276, 2) (2,) (2,) tf.Tensor([-0.96875 -0.96875], shape=(2,), dtype=float32) tf.Tensor([5 5], shape=(2,), dtype=uint8)
3 (2, 1, 1, 12, 3276, 2) (2, 1, 8, 12, 3276, 2) (2,) (2,) tf.Tensor([-0.96875 -0.96875], shape=(2,), dtype=float32) tf.Tensor([5 5], shape=(2,), dtype=uint8)
4 (2, 1, 1, 12, 3276, 2) (2, 1, 8, 12, 3276, 2) (2,) (2,) tf.Tensor([1.390625 1.390625], shape=(2,), dtype=float32) tf.Tensor([5 5], shape=(2,), dtype=uint8)
5 (2, 1, 1, 12, 3276, 2) (2, 1, 8, 12, 3276, 2) (2,) (2,) tf.Tensor([1.390625 1.390625], shape=(2,), dtype=float32) tf.Tensor([5 5], shape=(2,), dtype=uint8)
6 (2, 1, 1, 12, 3276, 2) (2, 1, 8, 12, 3276, 2) (2,)

In [ ]:
data_dir = f'../Dataset/{datetime.now().strftime("%Y-%m-%d")}/{datetime.now().strftime("%H-%M-%S")}'
start = time.time()

mcss = [5,6,7,8,9]
cdl_models = ["A", "C"]
speeds = [10]
delay_spreads = [150e-9]

PUSCH_SIMS = {
    "bler" : [],
    "ber" : [],
    }

num_cases = len(cdl_models) * len(speeds) * len(delay_spreads) * len(mcss)
# loop different channel models, speeds, delay spreads, MCS levels etc.
for (mcs, cdl_model, speed, delay_spread) in \
        (pbar := tqdm(itertools.product(mcss, cdl_models, speeds, delay_spreads), total=num_cases)):
    status_str = f"Generating... (MCS: {mcs} | CDL-{cdl_model} | {speed} m/s | {delay_spread} s)"
    pbar.set_description(status_str)
    channel_model = CDL(cdl_model, delay_spread, 2.55e9, ue_antenna, gnb_array, "uplink", min_speed=speed)
    channel_dir = f'{data_dir}/CDL{cdl_model}_speed{speed}_ds{delay_spread*1e9}'

    pusch_config_i = MyPUSCHConfig()
    pusch_config_i.tb.mcs_index = mcs
    print(channel_dir)
    model = MyGenerator(pusch_config_i, channel_model, mode=0, dir=channel_dir)
    ber, bler = sim_ber(model,
                np.arange(-10,-8.4,0.5),
                batch_size=2,
                max_mc_iter=2,
                num_target_block_errors=4,
                early_stop=False)
    PUSCH_SIMS["ber"].append(list(ber.numpy()))
    PUSCH_SIMS["bler"].append(list(bler.numpy()))

  0%|          | 0/10 [00:00<?, ?it/s]

../Dataset/2025-02-13/10-13-52/CDLA_speed10_ds150.0
EbNo [dB] |        BER |       BLER |  bit errors |    num bits | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
    -10.0 | 0.0000e+00 | 0.0000e+00 |           0 |      116768 |            0 |           4 |         7.9 |reached max iter       
     -9.5 | 0.0000e+00 | 0.0000e+00 |           0 |      116768 |            0 |           4 |         7.4 |reached max iter       
     -9.0 | 0.0000e+00 | 0.0000e+00 |           0 |      116768 |            0 |           4 |         7.2 |reached max iter       
     -8.5 | 0.0000e+00 | 0.0000e+00 |           0 |      116768 |            0 |           4 |         7.7 |reached max iter       
../Dataset/2025-02-13/10-13-52/CDLC_speed10_ds150.0
EbNo [dB] |        BER |       BLER |  bit errors |    num bits | block errors |  num blocks | runtime [s] |    s

KeyboardInterrupt: 

In [ ]:
data_dir = f'../Dataset/{datetime.now().strftime("%Y-%m-%d")}/{datetime.now().strftime("%H-%M-%S")}'
save_to_dataset = True
start = time.time()
caseIdx = 0


num_cases = len(channel_models) * len(esnos) * len(speeds) * len(delay_spreads) * len(mcss)
num_samples_per_param = num_samples // num_cases

# loop different channel models, speeds, delay spreads, MCS levels etc.
for (channel_model, esno, speed, delay_spread, mcs) in \
        (pbar := tqdm(itertools.product(channel_models, esnos, speeds, delay_spreads, mcss), total=num_cases)):

    status_str = f"Generating... ({channel_model} | {esno} dB | {speed} m/s | {delay_spread} s | MCS {mcs})"
    pbar.set_description(status_str)
            
            caseIdx += 1
            channel_dir = f'{data_dir}/CDL{cdl_model}{int(delay_spread*1e9)}-{speed}'
            
            for n,rbsize in enumerate(PUSCH_SIMS['RBSize']):
                for mcs in PUSCH_SIMS['MCS'][n]:

                    pusch_config_i = pusch_config.clone()
                    pusch_config_i.n_size_bwp = rbsize
                    pusch_config_i.tb.mcs_index = mcs
                    dir = f'{channel_dir}/MCS_{mcs}-{rbsize}_RBs'
                    print(dir)
                    model = Generator(pusch_config_i, channel_model, mode=0, dir=dir)
                    model.save_to_dataset = save_to_dataset
                    ber, bler = sim_ber(model,
                                np.arange(-8,-4.9,0.5),
                                batch_size=8,
                                max_mc_iter=4,
                                num_target_block_errors=32,
                                early_stop=False)
                    PUSCH_SIMS["ber"].append(list(ber.numpy()))
                    PUSCH_SIMS["bler"].append(list(bler.numpy()))
                    
PUSCH_SIMS["duration"] = time.time() - start

IndentationError: unexpected indent (1710936668.py, line 17)

Hopefully you have enjoyed this tutorial on Sionna's 5G NR PUSCH module!

Please have a look at the [API documentation](https://nvlabs.github.io/sionna/api/sionna.html) of the various components or the other available [tutorials](https://nvlabs.github.io/sionna/tutorials.html) to learn more.